<H1>Marto Teórico:Algoritmos de Boosting: Gradient Boosting y XGBoost</H1>

### **1. Introducción a Boosting**

**Boosting** es un método de ensamblaje que construye un fuerte predictor combinando varios modelos débiles en un proceso secuencial. Cada modelo en la secuencia corrige los errores del modelo anterior.

#### **1.1. Fundamentos de Boosting**

- **Modelo Débil**: Un modelo que realiza predicciones ligeramente mejor que el azar. En boosting, se suelen usar árboles de decisión pequeños, llamados *stumps*.
- **Modelo Fuerte**: El modelo combinado resultante de sumar los modelos débiles.

#### **1.2. Proceso General de Boosting**

1. **Inicialización**: Comienza con un modelo base simple, por ejemplo, el modelo de media en regresión o la predicción de la clase mayoritaria en clasificación.
2. **Iteración**:
   - Ajusta el modelo a los errores residuales del modelo anterior.
   - Actualiza las ponderaciones de los datos para enfocarse más en los ejemplos mal clasificados.
3. **Predicción Final**: La predicción final es una combinación ponderada de las predicciones de todos los modelos.

### **2. Gradient Boosting**

**Gradient Boosting** construye modelos secuenciales donde cada nuevo modelo trata de corregir los errores del modelo anterior, utilizando el descenso por gradiente para minimizar la función de pérdida.

#### **2.1. Conceptos Clave de Gradient Boosting**

- **Función de Pérdida**: Es la medida que se intenta minimizar. Para regresión, una función de pérdida común es el error cuadrático medio (MSE), y para clasificación, se usa la log-loss (entropía cruzada).

  Para regresión:
  
  $
  L(y, \hat{y}) = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2
  $
  
  Para clasificación:
  
  $
  L(y, \hat{y}) = -\frac{1}{N} \sum_{i=1}^{N} \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right]
 $

- **Modelo Base**: En Gradient Boosting, comúnmente se utilizan árboles de decisión poco profundos. El objetivo es ajustar estos árboles para minimizar el error residual en cada iteración.

- **Actualización del Modelo**: En cada iteración $m$, el modelo se actualiza como sigue:

  $
  f_{m+1}(x) = f_m(x) + \eta \cdot h_m(x)
  $

  Donde:
  - $f_m(x)$ es la predicción del modelo en la m-ésima iteración.
  - $\eta$ es la tasa de aprendizaje (un hiperparámetro que controla el tamaño del paso en la optimización).
  - $h_m(x)$ es el nuevo modelo ajustado a los errores residuales.

- **Cálculo de Residuales**: Los residuales $r_i$  en la iteración $m$ se calculan como:

  $
  r_i = - \frac{\partial L(y_i, \hat{y}_i)}{\partial \hat{y}_i}
  $

  Para la pérdida cuadrática, esto se simplifica a:

  $
  r_i = y_i - \hat{y}_i
  $

### **3. XGBoost**

**XGBoost (Extreme Gradient Boosting)** es una extensión de Gradient Boosting que incluye mejoras en términos de rendimiento y eficiencia.

#### **3.1. Cálculo de Gradientes y Hessianos**

En XGBoost, se optimiza la función objetivo mediante el descenso por gradiente. Cada iteración del modelo se ajusta en función del gradiente y del hessiano de la función de pérdida.

##### **3.1.1. Gradientes**

Para cada dato $i$, el gradiente $g_i$ de la función de pérdida $L$ con respecto a la predicción $\hat{y}_i$ es:

$
g_i = \frac{\partial L(y_i, \hat{y}_i)}{\partial \hat{y}_i}
$

Donde $y_i$  es el valor verdadero y $\hat{y}_i$ es la predicción del modelo.

**Ejemplo para la pérdida cuadrática (regresión):**

$
L(y_i, \hat{y}_i) = (y_i - \hat{y}_i)^2
$

El gradiente es:

$
g_i = -2 (y_i - \hat{y}_i)
$

**Ejemplo para la log-loss (clasificación):**

$
L(y_i, \hat{y}_i) = - \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right]
$

El gradiente es:
$
g_i = \hat{y}_i - y_i
$

##### **3.1.2. Hessianos**

El hessiano $h_i$ es la segunda derivada de la función de pérdida con respecto a la predicción $\hat{y}_i$:

$
h_i = \frac{\partial^2 L(y_i, \hat{y}_i)}{\partial \hat{y}_i^2}
$

**Ejemplo para la pérdida cuadrática (regresión):**

$
h_i = 2
$

**Ejemplo para la log-loss (clasificación):**

$
h_i = \hat{y}_i (1 - \hat{y}_i)
$

##### **3.1.3. Suma de Gradientes y Hessianos**

Para cada nodo del árbol, se calcula la suma total de los gradientes \( G \) y de los hessianos \( H \) de los datos en ese nodo.

- **Suma de Gradientes:**

  $
  G = \sum_{i \in \text{nodo}} g_i
  $

- **Suma de Hessianos:**

  $
  H = \sum_{i \in \text{nodo}} h_i
  $

#### **3.2. Ganancia de Información y Regularización**

En XGBoost, la ganancia de una división en el árbol se calcula para decidir si una división es beneficiosa. La ganancia \( \text{Gain} \) se calcula como:

$
\text{Gain} = \frac{1}{2} \left( \frac{(G_L^2)}{H_L + \lambda} + \frac{(G_R^2)}{H_R + \lambda} - \frac{(G^2)}{H + \lambda} \right) - \gamma
$

Donde:
- $ G_L $ y $ H_L $ son la suma de gradientes y hessianos en el nodo izquierdo $ L $ después de la división.
- $ G_R $ y $ H_R $ son la suma de gradientes y hessianos en el nodo derecho $ R $ después de la división.
- $ G $ y $ H $ son la suma total de gradientes y hessianos antes de la división.
- $ \lambda $ es el parámetro de regularización L2.
- $ \gamma $ es el parámetro de regularización que representa la ganancia mínima necesaria para realizar una división.

#### **3.3. Cálculo de Pesos en los Árboles**

El peso del nodo en XGBoost se calcula usando los gradientes y hessianos para minimizar la función de pérdida. Para un nodo hoja, el peso $ w $ se calcula como:

$
w = -\frac{G}{H + \lambda}
$

Donde:
- $ G $ es la suma de gradientes en el nodo hoja.
- $ H $ es la suma de hessianos en el nodo hoja.
- $ \lambda $ es el parámetro de regularización L2.

#### **3.4. Actualización de Predicciones**

En cada iteración, la predicción del modelo se actualiza añadiendo una nueva función ajustada:

$
\hat{y}_{m+1} = \hat{y}_m + \eta \cdot h_m(x)
$

Donde:
- $\eta$ es la tasa de aprendizaje.
- $h_m(x)$  es la predicción del m-ésimo árbol.

### **4. Comparación de Bagging y Boosting**

**Bagging** y **Boosting** son técnicas de ensamblaje que mejoran la precisión de los modelos, pero se diferencian en su enfoque.

#### **4.1. Bagging**

- **Enfoque**: Reduce la varianza entrenando múltiples modelos independientes y promediando sus predicciones.

- **Método**: Usa muestreo con reemplazo para crear múltiples subconjuntos de datos. La predicción final es el promedio (para regresión) o la votación mayoritaria (para clasificación) de todas las

 predicciones.

  $
  \hat{y}_{\text{bagging}} = \frac{1}{M} \sum_{m=1}^{M} \hat{y}_m
  $

  Donde $\hat{y}_m $ es la predicción del m-ésimo modelo, y $ M $ es el número total de modelos.

#### **4.2. Boosting**

- **Enfoque**: Reduce el sesgo ajustando los modelos secuencialmente para corregir los errores del modelo anterior.

- **Método**: Cada nuevo modelo se ajusta a los errores residuales del modelo anterior y se actualiza en función del gradiente de la función de pérdida.

  $
  \hat{y}_{\text{boosting}} = f_0(x) + \sum_{m=1}^{M} \eta \cdot h_m(x)
  $

  Donde $f_0(x)$ es el modelo base y $\eta$ es la tasa de aprendizaje.


<H1>Módulo 8: Algoritmos de Boosting: Gradient Boosting y XGBoost </H1>

**Conceptos clave:**

Introducción a boosting.

Gradient Boosting y XGBoost.

Comparación de bagging y boosting.

Optimización de hiperparámetros con GridSearchCV.

Proyecto: Predicción de abandono de clientes (Churn).
Utilizar un dataset de telecomunicaciones para predecir el churn (abandono de clientes) utilizando Gradient Boosting y XGBoost.

**Módulo 8: Algoritmos de Boosting: Gradient Boosting y XGBoost**

**Conceptos clave**
Introducción a Boosting:

Boosting es una técnica de aprendizaje automático que combina múltiples modelos débiles (como árboles de decisión simples) para crear un modelo fuerte. Cada nuevo modelo se ajusta a los errores del modelo anterior, lo que mejora la precisión del modelo final.
Gradient Boosting y XGBoost:

Gradient Boosting: Un algoritmo de boosting que optimiza una función de pérdida mediante la adición de modelos secuenciales. Cada modelo nuevo corrige los errores del anterior.

XGBoost (Extreme Gradient Boosting): Una variante eficiente y escalable de Gradient Boosting que incluye técnicas adicionales para mejorar la precisión y reducir el tiempo de entrenamiento.

**Comparación de Bagging y Boosting:**

Bagging: Mejora la estabilidad y precisión de los modelos combinando predicciones de múltiples modelos entrenados en subconjuntos de datos diferentes.

Boosting: Enfocado en mejorar el rendimiento de modelos secuenciales, ajustando errores del modelo anterior.

**Optimización de hiperparámetros con GridSearchCV:**

GridSearchCV es una técnica para encontrar la mejor combinación de hiperparámetros para un modelo al evaluar un conjunto de combinaciones posibles y seleccionar la que da el mejor rendimiento.

**Proyecto: Predicción de abandono de clientes (Churn)**

**Paso 1: Importar librerías y cargar el dataset**

Primero, importamos las librerías necesarias y cargamos el dataset desde la web.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

# Cargar el dataset desde la web
url = '/home/julio/jupyter_files/Curso_ML/WA_Fn-UseC_-Telco-Customer-Churn.csv'
df = pd.read_csv(url)

# Mostrar las primeras filas del dataset
print('=================================================================')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


**Paso 2: Exploración y Preprocesamiento de Datos**

Exploramos el dataset y preprocesamos los datos (manejo de valores faltantes, codificación de variables categóricas, etc.).

In [7]:
# Separar características y variable objetivo
X = df.drop('Churn', axis=1)
y = df['Churn'].map({'Yes': 1, 'No': 0})  # Convertir 'Churn' a valores numéricos

# Identificar columnas categóricas y numéricas
categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns

# Definir el preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numerical_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ]
)

# Crear pipeline para Gradient Boosting
gb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier())
])

# Crear pipeline para XGBoost
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
])


**Paso 3: Entrenamiento y Evaluación del Modelo**

Usamos los pipelines para entrenar y evaluar los modelos.

In [8]:
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
# Entrenar y evaluar el modelo de Gradient Boosting
gb_pipeline.fit(X_train, y_train)
y_pred_gb = gb_pipeline.predict(X_test)

print("Gradient Boosting Classification Report:")
print(classification_report(y_test, y_pred_gb))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_gb))

Gradient Boosting Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.92      0.87      1539
           1       0.69      0.50      0.58       574

    accuracy                           0.81      2113
   macro avg       0.76      0.71      0.73      2113
weighted avg       0.79      0.81      0.79      2113

Confusion Matrix:
[[1412  127]
 [ 285  289]]


In [10]:
# Entrenar y evaluar el modelo de XGBoost
xgb_pipeline.fit(X_train, y_train)
y_pred_xgb = xgb_pipeline.predict(X_test)

print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_xgb))

/home/julio/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:07:45] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.88      0.86      1539
           1       0.62      0.53      0.57       574

    accuracy                           0.78      2113
   macro avg       0.73      0.70      0.71      2113
weighted avg       0.78      0.78      0.78      2113

Confusion Matrix:
[[1355  184]
 [ 271  303]]


**Paso 4: Optimización de Hiperparámetros con GridSearchCV**

Aplicamos GridSearchCV para encontrar los mejores hiperparámetros.

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# Definir el rango de hiperparámetros para Gradient Boosting
gb_param_grid = {
    'classifier__n_estimators': [50, 100, 150],
    'classifier__learning_rate': [0.01, 0.1, 0.2],
    'classifier__max_depth': [3, 4, 5]
}

In [13]:
# Configurar GridSearchCV para Gradient Boosting
gb_grid_search = GridSearchCV(estimator=gb_pipeline, param_grid=gb_param_grid, cv=3, scoring=make_scorer(lambda y_true, y_pred: np.mean(y_true == y_pred)))
gb_grid_search.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['SeniorCitizen', 'tenure', 'MonthlyCharges'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEn...
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod', 'TotalCharges'],
      dtype='object'))])),
                                       ('classifier',
                                        GradientBoostingClassifier())]),
             param_grid={'classifier__learning_rate': [0.01, 0.1, 0.2],
                         'classifier__max_depth': [3, 4, 5],
                         'classifier__n_estimators': [50, 100, 150]},
             scoring=make_scorer(<lambda>, response_method='predict'))

In [14]:
# Evaluar el modelo con los mejores hiperparámetros
y_pred_gb_best = gb_grid_search.best_estimator_.predict(X_test)

In [15]:
# Mostrar los resultados
print("Optimized Gradient Boosting:")
print(f"Best parameters: {gb_grid_search.best_params_}")
print(f"Classification Report:\n{classification_report(y_test, y_pred_gb_best)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred_gb_best)}")

Optimized Gradient Boosting:
Best parameters: {'classifier__learning_rate': 0.2, 'classifier__max_depth': 5, 'classifier__n_estimators': 150}
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.91      0.87      1539
           1       0.68      0.53      0.60       574

    accuracy                           0.81      2113
   macro avg       0.76      0.72      0.73      2113
weighted avg       0.80      0.81      0.80      2113

Confusion Matrix:
[[1394  145]
 [ 267  307]]


In [16]:
# Definir el rango de hiperparámetros para XGBoost
xgb_param_grid = {
    'classifier__n_estimators': [50, 100, 150],
    'classifier__learning_rate': [0.01, 0.1, 0.2],
    'classifier__max_depth': [3, 4, 5],
    'classifier__subsample': [0.8, 0.9, 1.0]
}

In [17]:
# Configurar GridSearchCV para XGBoost
xgb_grid_search = GridSearchCV(estimator=xgb_pipeline, param_grid=xgb_param_grid, cv=3, scoring=make_scorer(lambda y_true, y_pred: np.mean(y_true == y_pred)))
xgb_grid_search.fit(X_train, y_train)

/home/julio/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:15:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/julio/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:15:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/julio/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:15:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/julio/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:15:04] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/julio/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:15:05] WARNING: /workspace/src/lea

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['SeniorCitizen', 'tenure', 'MonthlyCharges'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEn...
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      multi_strategy=None,
                                                      n_estimators=None,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      random_state=None, ...))]),
             param_grid={'classifier__learning_rate': [0.01, 0.1, 0.2],
                         'classifier__max_depth': [3, 4, 5],
                         'classifier__n_estimators': [50, 100, 150],
                         'classifier__subsample': [0.8, 0.9, 1.0]},
             scoring=make_scorer(<lambda>, response_method='predict'))

In [18]:
# Evaluar el modelo con los mejores hiperparámetros
y_pred_xgb_best = xgb_grid_search.best_estimator_.predict(X_test)

In [19]:
# Mostrar los resultados
print("Optimized XGBoost:")
print(f"Best parameters: {xgb_grid_search.best_params_}")
print(f"Classification Report:\n{classification_report(y_test, y_pred_xgb_best)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred_xgb_best)}")

Optimized XGBoost:
Best parameters: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 100, 'classifier__subsample': 1.0}
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.91      0.87      1539
           1       0.68      0.54      0.60       574

    accuracy                           0.81      2113
   macro avg       0.76      0.72      0.74      2113
weighted avg       0.80      0.81      0.80      2113

Confusion Matrix:
[[1396  143]
 [ 264  310]]


**Paso 5: Conclusiones**

Comparamos los resultados optimizados sin mantener grandes volúmenes de datos en memoria.

In [20]:
print("Resumen de resultados:")
print(f"Gradient Boosting - Best Score: {gb_grid_search.best_score_:.2f}")
print(f"XGBoost - Best Score: {xgb_grid_search.best_score_:.2f}")

Resumen de resultados:
Gradient Boosting - Best Score: 0.80
XGBoost - Best Score: 0.80
